In [5]:
import nltk
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from textUtils import cleanAll, getTFIDF_vectors, getVector
from datetime import datetime as date
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ilove\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
df = pd.read_csv("BestDataset2.csv")
df = df.dropna(subset=['description', 'bedrooms_count', 'average_rate_per_night'])
df = df.drop_duplicates(subset=['title'])
df = df.drop_duplicates(subset=['url'])
df = df.drop(df[df['description'].str.len() < 5].index)
# df['bedrooms_count'] = df['bedrooms_count'].str.replace('Studio', '1')
df['average_rate_per_night'] = df['average_rate_per_night'].str.replace('$', '')
df['average_rate_per_night'] = df['average_rate_per_night'].str.replace(',', '')
# df['title'] = df['title'].fillna('a')
# df['average_rate_per_night'] = df['average_rate_per_night'].fillna(df['average_rate_per_night'].mean())

cleanedTitle = df['title'].apply(cleanAll)
cleanedDescription = df['description'].apply(cleanAll)

corpusTitle = [words.split() for words in cleanedTitle]
corpusDescriptin = [words.split() for words in cleanedDescription]
google_model = Word2Vec(size=300, window=5, min_count=5, workers=6)
google_model.build_vocab(corpusTitle + corpusDescriptin)

google_model.intersect_word2vec_format('./data/GoogleNews-vectors-negative300.bin', lockf=1.0, binary=True)
google_model.train(corpusTitle + corpusDescriptin, total_examples=google_model.corpus_count, epochs=5)

print("loaded google model")

print("description loading: " + str(date.now()))
tfidf_featureDescription, tfidf_listDescription, tfidf_vectorsDescription = getTFIDF_vectors(cleanedDescription,corpusDescriptin, google_model)
print("\ntitle loading: " + str(date.now()))
tfidf_featureTitle, tfidf_listTitle, tfidf_vectorsTitle = getTFIDF_vectors(cleanedTitle,corpusTitle, google_model)
print("\ncreated vectors")

loaded google model
description loading: 2021-01-09 12:53:26.753815
77729/77729
title loading: 2021-01-10 00:13:42.921346
77729/77729
created vectors


In [7]:
import pickle
PIK = "data/pickle.dat"

data = [google_model,[tfidf_featureDescription, tfidf_listDescription, tfidf_vectorsDescription],[tfidf_featureTitle, tfidf_listTitle, tfidf_vectorsTitle]]
with open(PIK, "wb") as f:
    pickle.dump(data, f)

In [9]:
# description = "I want a quiet and warm place with a pool next to a beach, perfect for a couple."
description = "I wanna go to big house close to a mountain"
description = cleanAll(description)

desc = description.split()
sent_vecDescription = getVector(desc,tfidf_featureDescription,tfidf_listDescription, google_model)
sent_vecTitle = getVector(desc,tfidf_featureTitle,tfidf_listTitle, google_model)
cosine_similaritiesDescription = cosine_similarity([sent_vecDescription],tfidf_vectorsDescription)
cosine_similaritiesTitle = cosine_similarity([sent_vecTitle],tfidf_vectorsTitle)

sim_scoresDescription = list(enumerate(cosine_similaritiesDescription[0]))
sim_scoresDescription = [x if x[1] <=1 else None for x in sim_scoresDescription]

sim_scoresTitle = list(enumerate(cosine_similaritiesTitle[0]))
sim_scoresTitle = [x if x[1] <=1 else None for x in sim_scoresTitle]

sim_scores = [(sd[0],sd[1] + st[1] * 0.8) for sd,st in zip(sim_scoresDescription,sim_scoresTitle)]
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[0:10]
places_indices = [i[0] for i in sim_scores]

places = df[['description','title', 'url', 'average_rate_per_night', 'bedrooms_count', 'city']]
recommend = places.iloc[places_indices]
for index, row in recommend.iterrows():
    print(row['title']+": "+row['description'])



Amazing view, comfortable/big room: The house is brand new and the room is finely decorated ! The room is very large ( 20m2 ), with a big bathroom and an amazing view of the Redeemer ! The house location is great , in a private condominium in the middle of South Zone of Rio de , with 24h security . We are close to , Botanical Garden , , and , with nearby / . The house in a green area . We are often by beautiful , and . Our house is in a hill in front of the , , the Botanical Garden and the Forest . The room is very large and and the bathroom is newly . We speak , and . We will offer you all the information you need to have a marvelous experience in town . We are close to many . There are great , , shopping and famous nearby . For those who enjoy physical , there is a possibility to practice sports in . T
2 Bedrooms, mountain view, Close to Beach & Subway: Beautiful high ceiling apt w 2 , Living & study looking to Rio ' s striking mountain landscape . Warm , Colorful & Sunny , decorated